# main.ipynb

In [ ]:
from utils import *
from PIVAnalysis import *

In [ ]:
# Get datasets directory
folder_path = Path.cwd().parent.parent / 'datasets' / 'piv' / 'background_subtracted_trial' 
file_list = os.listdir(folderPath)

# Read image
i = 0
image_path = folder_path / file_list[i]
movie = imageio.imread(image_path)

## Parameters for PIV

In [ ]:
# -- Pre-processing
WINSIZE:int = 32 # pixels, interrogation window size in frame 't'
SEARCHSIZE:int = 38  # pixels, search area size in frame 't+1'
OVERLAP:int = 17 # pixels, 50% overlap
DT:float = 0.25 # sec, time interval between the two frames
PX2UM:float = 2.75 # pixels/micrometers
# -- Post-processing
S2N_THRESHOLD:float = 1.05 # AU, signal-to-noise threshold to remove false values

In [ ]:
# Test movie
# a subset of 300x300 pixels on the top left corner of the original image

test_movie = movie[:300, :300]

frame_a = adjust_contrast(test_movie[0,:,:])
frame_b = adjust_contrast(test_movie[1,:,:])

check if piv_image2image is working
- display t & t+1
- display resulting vectors

In [ ]:
x, y, u, v = piv_image2image(
    frame_a, 
    frame_b, 
    WINSIZE, 
    SEARCHSIZE, 
    OVERLAP, 
    DT, 
    PX2UM, 
    S2N_THRESHOLD)

# Plot
fig, ax = plt.subplots(3, 1, figsize=(10, 10))
ax[0].imshow(frame_a, cmap='gray')
ax[1].imshow(frame_b, cmap='gray')
ax[2].quiver(x, y, u, v, color='r')

plt.show()

### Effect of parameters

effect of window size on the same image subset
- 16 px
- 32 px
- 64 px
CONSTANT PARAMETERS = overlap stays at 50%

In [ ]:
test_windsize = [16, 32, 64]

for index, value in enumerate(test_windsize):
    x, y, u, v = piv_image2image(
        frame_a, 
        frame_b, 
        value, 
        SEARCHSIZE, 
        OVERLAP, 
        DT, 
        PX2UM, 
        S2N_THRESHOLD)

    # Plot
    fig, ax = plt.subplots(max(index), 1, figsize=(10, 10))
    ax[index].imshow(frame_a, cmap='gray')
    ax[index].quiver(x, y, u, v, color='r')

    plt.show()

effect of frame skipping on the same image subset
- 1:1
- 1:2
- 1:4
- 1:8
CONSTANT PARAMETERS = S2N at 1.05

In [ ]:
test_windsize = [16, 32, 64]

for index, value in enumerate(test_windsize):
    x, y, u, v = piv_image2image(
        frame_a, 
        frame_b, 
        value, 
        SEARCHSIZE, 
        OVERLAP, 
        DT, 
        PX2UM, 
        S2N_THRESHOLD)

    # Plot
    fig, ax = plt.subplots(max(index), 1, figsize=(10, 10))
    ax[index].imshow(frame_a, cmap='gray')
    ax[index].quiver(x, y, u, v, color='r')

    plt.show()

effect of S2N
- 1.05
- 1.025
- 1.1
- 1.3

combined effect of S2N + timeskip

## Mean values

compute over the film
- norm
- divergence
- vorticity

In [ ]:
flow_norm, flow_divergence, flow_vorticity = movie_flow_features(x, y, u, v, start=0, end=T)

# Median values on 2D map over the course of the film
fig, ax = plt.subplots(1, 3, figsize=(15,5))

# Norm
mean_flow_norm = np.median(flow_norm, axis=0)
im0 = ax[0].imshow(mean_flow_norm, cmap='viridis')
ax[0].set_title('Median Flow Norm')
plt.colorbar(im0, ax=ax[0])

# Divergence
mean_flow_divergence = np.median(flow_divergence, axis=0)
im1 = ax[1].imshow(mean_flow_divergence, cmap='bwr', norm=CenteredNorm())
ax[1].set_title('Median Flow Divergence')
plt.colorbar(im1, ax=ax[1])

# Vorticity
mean_flow_vorticity = np.median(flow_vorticity, axis=0)
im2 = ax[2].imshow(mean_flow_vorticity, cmap='bwr', norm=CenteredNorm())
ax[2].set_title('Median Flow Vorticity')
plt.colorbar(im2, ax=ax[2])

plt.tight_layout()
plt.show()